All information needed will be added in this code, each code has a markdown on top of it for an explanation

Some libraries are added

In [ ]:
from bs4 import BeautifulSoup as bs

import os, glob

import shutil
from IPython.utils.io import capture_output
import random

#from my_simulation import simulacion  

#This is the only line for the directory you should need to change
dir_tutti="/home/jazmin/tuttifrutti"



First we add the activities of the mision, and the probability of success

Go ahead and select option for a random collection of activities or use the example given (modify if like, the code will still work)


In [ ]:
#use the word "manual" or "random"
mision_activities="random"

#probability from 0 to 1
fixed_prob="probability=0.8;"

#number of times the mision is repeated to be stored in one log
repetition_numer=10

#duration of the simulation in seconds
timer=20000


Here we define the activities, this function will be called, if you selected a "manual" choice of the activities, here you can modify it

In [ ]:
 

def activities():
    if mision_activities=="manual":   
        ################################ ADD THE ACTIVITIES #######################################################     #delete previos content
        file=dir_tutti+"/log/Tesis/mision_variables/activities.txt"
        with open(file, "w") as f:
            f.write("activities[\"0_sec\"] = {0,22,23};")
            f.write("\n") 
            f.write("activities[\"1_con\"] = {31,25,26};")

            ################################ ADD THE ACTIVITIES #######################################################

    if mision_activities=="random":
        numbers_total = list(range(2, 30 + 1))
        numbers_list=list(range(2, 30 + 1))
        weights_t = [1] * len(numbers_total)
        weights_l = [1] * len(numbers_list)

        max_total_values = random.choices(numbers_total, weights_t)[0]
        print(f"max: {max_total_values}" )


        sum_total = 0
        dictionary_lists = {}

        list_counter = 0
        initial_word = random.choice(["con\"", "sec\""])
        previous_word = initial_word

        while sum_total != max_total_values:
            # Genera una lista de números sin repetición
            minimo=min(30, max_total_values - sum_total)
            if (minimo==1):
                break
            numbers_random = list(range(2, min( 30, max_total_values - sum_total) + 1))
            # Asigna el mismo peso a cada número (en este caso, 1)
            weights_n = [1] * len(numbers_random)

            # Elige un número aleatorio con igual probabilidad
            random_number = random.choices(numbers_random, weights_n)[0]

            if sum_total + random_number <= max_total_values:
                sum_total += random_number
            else:
                continue

            if previous_word == "con\"":
                current_word = "sec\""
            else:
                current_word = "con\""

            key = f"\"{list_counter}_{current_word}"

            # Genera números aleatorios con igual probabilidad utilizando random.choices
            if current_word == "con\"":
                # Crear población excluyendo valores ya seleccionados en "con\""
                population = set(range(60)) - set(dictionary_lists.get(f"\"{list_counter}_con\"", []))
                random_numbers = random.choices(list(population), k=random_number)
            else:
                random_numbers = random.choices(range(60), k=random_number)

            dictionary_lists[key] = random_numbers
            list_counter += 1
            previous_word = current_word



        print("Total sum:", sum_total)
        print("Dictionary of lists:")

        file=dir_tutti+"/log/Tesis/mision_variables/activities.txt"
        with open(file, "a") as f:        
            for key, lista in dictionary_lists.items():
                print(f'{key}: {lista}')  
                lista_str = ', '.join(map(str, lista))
                f.write("\n") 
                f.write("activities[" + key + "] = {" + lista_str + "};")



execute the mision, you can change variable n_repetition 

Next line is executed when option manual is used, change the folder path to your needs


you can change the number of repetitions of the mision.

In [ ]:
file=dir_tutti+"/log/Tesis/mision_variables/activities.txt"
file_send=dir_tutti+"/log/Tesis/EventLogs/"
with open(file, "w") as f:
    f.write("probability=1;")
activities()
!python /home/jazmin/tuttifrutti/log/Tesis/Codigos/seed_change_multiple_times.py --directorio $file_send --repetition $repetition_numer  --time $timer

In [ ]:
file=dir_tutti+"/log/Tesis/mision_variables/activities.txt"
file_send=dir_tutti+"/log/Tesis/EventLogs_fail/"
with open(file, "r") as archivo:
    lineas = archivo.readlines()
lineas[0] = fixed_prob+"\n"
with open(file, "w") as archivo:
    archivo.writelines(lineas)
!python /home/jazmin/tuttifrutti/log/Tesis/Codigos/seed_change_multiple_times.py --directorio $file_send --repetition $repetition_numer  --time $timer

Store all information for the mision in one folder

1. Create new folder

In [ ]:
# Directory where you want to search for folders
directory = dir_tutti+"/log/Tesis/"

# Prefix that should be in the folder names
prefix = "example"

# Find all folders with the "example" prefix in the name
folders = [name for name in os.listdir(directory) if prefix in name]

# Find the highest number in the folder names
numbers = [int(name.replace(prefix, "")) for name in folders]
highest_number = max(numbers) if numbers else 0

# Create the name for the new folder
new_name = f"{prefix}{highest_number + 1}"

# Full path of the new folder
new_folder = os.path.join(directory, new_name)

Code to copy folder into the new folder

In [ ]:
def copy_folder(source, destination):
    try:
        # Create the new folder if it doesn't exist
        if not os.path.exists(destination):
            os.mkdir(destination)
        # Use the `copytree` function from shutil to copy the folder and its contents
        shutil.copytree(source, os.path.join(destination, os.path.basename(source)))
        print(f"Folder '{os.path.basename(source)}' copied to {destination}")
    except Exception as e:
        print(f"Unable to copy the folder: {e}")

Code to copy a file into a new folder

In [ ]:
def copy_file(source, destination):
    try:
        # Use the `copy` function from shutil to copy the file
        shutil.copy(source, destination)
        print(f"File copied from {source} to {destination}")
    except Exception as e:
        print(f"Unable to copy the file: {e}")


Files to copy

In [ ]:
event_log = dir_tutti+"/log/Tesis/EventLogs"
event_log_fail =dir_tutti+"/log/Tesis/EventLogs_fail"
variables=dir_tutti+"/log/Tesis/mision_variables/activities.txt"

copy_folder(event_log, new_folder)
copy_folder(event_log_fail, new_folder)
copy_file(variables, new_folder)

Next line is executed when option random is used, change the folder path to your needs

Rest of the time, executed from here

In [ ]:
import matplotlib.pyplot as plot
import pandas as pd
import numpy as np
import glob
import os
import seaborn as sns
from datetime import date
from IPython.display import Markdown, display
import matplotlib.pyplot as plt
import pm4py
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.visualization.petri_net import visualizer as pn_visualizer
from pm4py.objects.petri_net.utils.decomposition import decompose
from pm4py.objects.petri_net.utils import reachability_graph
from pm4py.visualization.transition_system import visualizer as ts_visualizer

Create one dataframe with the info from all EventLogs

In [ ]:

def concatenate(folder):
    filenames = glob.glob(folder + "/*.csv")
    dfs = []
    model=0
    for filename in filenames:
        log = pd.read_csv(filename, sep=',')
        case_id=[]
        case_id += len(log.mision) * ["mision "+str(model)]
        log['mision']=case_id
        model+=1
        dfs.append(log)
        big_frame = pd.concat(dfs, ignore_index=True)
        if model==len(filenames):
            big_frame.to_csv(dir_tutti+'/log/Tesis/LogFinal/final_log_2.csv', sep=',', index=False)


Set the names for the colums in both dataframes

In [ ]:
fn = dir_tutti+'/log/Tesis/LogFinal/final_log_2.csv'
concatenate(new_folder+"/EventLogs")
events = pd.read_csv(fn)
events.columns = ['mision', 'action', 'datetime', 'resource', 'random_seed']
events['datetime'] = pd.to_datetime(events['datetime'])
events


In [ ]:

concatenate(new_folder+"/EventLogs_fail")
events_fail = pd.read_csv(fn)
events_fail.columns = ['mision', 'action', 'datetime', 'resource', 'random_seed']
events_fail['datetime'] = pd.to_datetime(events_fail['datetime'])
events_fail

To use pm4py first we convert to an EventLog

In [ ]:
events.rename(columns={'datetime': 'time:timestamp', 'mision': 'case:concept:name', 'action': 'concept:name', 'random_seed': 'org:resource'}, inplace=True)
events_fail.rename(columns={'datetime': 'time:timestamp', 'mision': 'case:concept:name', 'action': 'concept:name', 'random_seed': 'org:resource'}, inplace=True)

## Convert to log format the events without fail probability
log_inicial = log_converter.apply(events)
## Convert to log format the events with fail probability
log_fail = log_converter.apply(events_fail)


For the ideal log, we don't need much information, just to make sure is perfect

We check for the variants

In [ ]:
variants = pm4py.get_variants(log_fail)
len(variants)
variants

There should be as many as number of (concurrent activities x number of actions on the activities) the mision has, if there's more, substrac them.

In [ ]:
k=3
log = pm4py.filter_variants_top_k(log_inicial, k)
log

In [ ]:
#log_no_timeout = pm4py.filter_activities_rework(log_inicial, "time_out", 1)
log_no_timeout = pm4py.filter_event_attribute_values(log_inicial, "concept:name", ["time_out"], level="case", retain=False)
log_no_timeout

We can get the model using different algorithims, we'll use 4: alpha miner, inductive miner, heuristic miner, lip miner

Heuristic miner

In [ ]:
#Petri net
net, im, fm = pm4py.discover_petri_net_heuristics(log_no_timeout, dependency_threshold=0.99)
parameters = {pn_visualizer.Variants.FREQUENCY.value.Parameters.FORMAT: "png"}
gviz = pn_visualizer.apply(net, im, fm , parameters=parameters, variant=pn_visualizer.Variants.FREQUENCY, log=log_no_timeout)
pn_visualizer.save(gviz, new_folder+"/Heuristic_miner_frequency_IDEAL.png")
#pm4py.view_petri_net(net, im, fm)

ILP miner

In [ ]:
net, im, fm = pm4py.discovery.discover_petri_net_ilp(log_inicial)
pm4py.view_petri_net(net, im, fm)
#parameters = {pn_visualizer.Variants.FREQUENCY.value.Parameters.FORMAT: "png"}
#gviz = pn_visualizer.apply(net, im, fm, parameters=parameters, variant=pn_visualizer.Variants.FREQUENCY, log=log_inicial)
#pn_visualizer.save(gviz, new_folder+"/ILP_miner_frequency_IDEAL.png")
#pm4py.view_petri_net(net, im, fm)

Inductivo miner

We'll use inductive miner at the end beause we'll use it for conformance checking and like this variables as net, initial_marking, final_marking are saved

In [ ]:
net, initial_marking, final_marking = pm4py.discover_petri_net_inductive(log_inicial)
pm4py.view_petri_net(net, initial_marking, final_marking)

#we can add frequency and performance, change the word to FREQUENCY or PERFORMANCE accordingly
#parameters = {pn_visualizer.Variants.FREQUENCY.value.Parameters.FORMAT: "png"}
#gviz = pn_visualizer.apply(net, initial_marking, final_marking, parameters=parameters, variant=pn_visualizer.Variants.FREQUENCY, log=log_inicial)
#pn_visualizer.save(gviz, new_folder+"/Inductive_miner_frequency_IDEAL.png")
#pn_visualizer.view(gviz)

In [ ]:
net, initial_marking, final_marking = pm4py.discover_petri_net_inductive(log_no_timeout)
pm4py.view_petri_net(net, initial_marking, final_marking)

#we can add frequency and performance, change the word to FREQUENCY or PERFORMANCE accordingly
#parameters = {pn_visualizer.Variants.PERFORMANCE.value.Parameters.FORMAT: "png"}
#gviz = pn_visualizer.apply(net, initial_marking, final_marking, parameters=parameters, variant=pn_visualizer.Variants.FREQUENCY, log=log_no_timeout)
#pn_visualizer.save(gviz, new_folder+"/Inductive_miner_frequency_IDEAL_not_time_out.png")
#pn_visualizer.view(gviz)

Now that we have an IDEAL model of the operation, we'll get some variables for the real proccess

REAL LOG

To only get activities that happen fast from 0 to 200 seconds only

In [ ]:

filtered_log = pm4py.filter_case_performance(log_fail, 0, 100)
filtered_log

Let's check the number of varians, the activities involved (with its frequency) and the resource (with its frequency)


Note: the resource can be exchange between robot and random_seed, to do so go back to the log converter and change the name of the resource


In [ ]:
variants = pm4py.get_variants(log_fail)
activities = pm4py.get_event_attribute_values(log_fail, "concept:name")
resources = pm4py.get_event_attribute_values(log_fail, "org:resource")
len(variants)
activities

If the probability of failure is too high you may enconter filtering the log to only the most k used variables useful

In [ ]:
k=len(variants)
log_variantes = pm4py.filter_variants_top_k(log_fail, k)

Now, lets say you you want to filter your log only on a especific attribute.
You can do it by case meaning all the cases that have that attribute, or by event by filtering only the events containing the attribute

In [ ]:
#You can get all the events containing a attribute
tracefilter_log_event = pm4py.filter_event_attribute_values(log_fail, "org:resource", [7], level="event", retain=False)
#Or all the cases containing a attribute
tracefilter_log_case = pm4py.filter_event_attribute_values(log_fail, "org:resource", [7], level="case", retain=False)


Now if we want to check for cases were two activities are follow by one anohter we use this filter, if you can check for example how many times an activity rebooted when x activity happend first, yo can do it here

In [ ]:
Log_between = pm4py.filter_between(log_fail, "T_0_Busy", "T_0_rebooting")
len(filtered_log)

To eliminate noise, we can eliminte logs that are too long (rework) or too short (incompleate)

In [ ]:
log_by_size = pm4py.filter_case_size(log_fail, 0, 9)
filtered_log

To check for activities being excecuted more than k time we use

In [ ]:
k=3
filtered_log = pm4py.filter_activities_rework(log_fail, 'T_0_rebooting', k)
print(filtered_log)

To check for paths that take too long we use paths performance, so we get the list of paths taking longer than expected

In [ ]:
filtered_log = pm4py.filter_paths_performance(log_fail, ("T_0_Done", "T_2_Busy"), 0, 200)
filtered_log

We can also model the REAL log, we'll use the same 4 algorithms, but we'll only save the inductive, since it works better and this models won't be used in conformance checking tecniches

INDUCTIVE MINER

This algorithim can be seen as a petri net, as a process tree and as a bpmn model.
According to the state of the art literature this alg should be the best at handeling the behaviour of a group of robots

In [ ]:
net_f, initial_marking_f, final_marking_f = pm4py.discover_petri_net_inductive(log_fail)

#we can add frequency and performance, change the word to FREQUENCY or PERFORMANCE accordingly
parameters = {pn_visualizer.Variants.PERFORMANCE.value.Parameters.FORMAT: "png"}
gviz = pn_visualizer.apply(net_f, initial_marking_f, final_marking_f, variant=pn_visualizer.Variants.FREQUENCY, log=log_fail)
pn_visualizer.view(gviz)
pn_visualizer.save(gviz, new_folder+"/Inductive_miner_frequency_REAL.png")


In [ ]:
tree = pm4py.discover_process_tree_inductive(log_fail)
pm4py.view_process_tree(tree)


In [ ]:
bpmn_graph = pm4py.discover_bpmn_inductive(log_fail)
#pm4py.view_bpmn(bpmn_graph)


    HEURISTIC MINER

    Heuristics Miner is an algorithm that acts on the Directly-Follows Graph, providing way to handle with noise and to find common constructs (dependency between two activities, AND).

    We will obtain a Heuristic net and a Petri Net

In [ ]:
#Heuristic net
heu_net = pm4py.discover_heuristics_net(log_fail, dependency_threshold=0.99)
#pm4py.view_heuristics_net(heu_net)

In [ ]:
#Petri net
net_f, initial_marking_f, final_marking_f= pm4py.discover_petri_net_heuristics(log_fail, dependency_threshold=0.99)
#pm4py.view_petri_net(net_f, initial_marking_f, final_marking_f)

ILP MINER

In [ ]:
net_f, initial_marking_f, final_marking_f = pm4py.discovery.discover_petri_net_ilp(log_fail)
#pm4py.view_petri_net(net_f, initial_marking_f, final_marking_f)


A great tool that allows us to check exactly activities that follow one another are directly follows diagrams, we have some bellow

In [ ]:
dfg, start_activities, end_activities = pm4py.discover_dfg(log_fail)
#pm4py.view_dfg(dfg, start_activities, end_activities)

If you want to add performance (average time) to the dfg

In [ ]:
performance_dfg, start_activities, end_activities = pm4py.discover_performance_dfg(log_fail)
#pm4py.view_performance_dfg(performance_dfg, start_activities, end_activities)

If the model is too complicated, but you want to analize it visually you can decompose it into smaller pieces, here you have the code

In [ ]:

net_f, initial_marking_f, final_marking_f = pm4py.discover_petri_net_inductive(log_fail)
list_nets = decompose(net_f, initial_marking_f, final_marking_f)
for index, model in enumerate(list_nets):
    subnet, s_im, s_fm = model
    #pm4py.view_petri_net(subnet, s_im, s_fm)

Great!
All that you just did is contamplated in process mining as process discovery, hope it was fun, there are many many other tools you can use, check the pm4py website if you want to explore them further.

Now we will be doing Conformance Checking, here we will use our REAL LOG and our IDEAL MODEL, we will compare them and if there are anomalies as we expect, we'll try to detect them

Conformance Checking

Token-based replay

 Based on this comparison, token replay provides insights into whether the recorded process adheres to the expected process model. It can identify deviations, missing steps, or extra steps in the actual process.

In [ ]:
import csv

# Create a list to store trace results
trace_results_list = []

replayed_traces = pm4py.conformance_diagnostics_token_based_replay(log_fail, net, initial_marking, final_marking)

# Iterate through the replayed traces and access conformance metrics for each trace
for i, trace_result in enumerate(replayed_traces, start=1):
    trace_is_fit = trace_result['trace_is_fit']  # Boolean indicating if the trace fits the model
    trace_fitness = trace_result['trace_fitness']  # Fitness value for the trace
    missing_tokens = trace_result['missing_tokens']  # Number of missing tokens
    consumed_tokens = trace_result['consumed_tokens']  # Number of consumed tokens
    remaining_tokens = trace_result['remaining_tokens']  # Number of remaining tokens
    produced_tokens = trace_result['produced_tokens']  # Number of produced tokens

    # Create a dictionary to store trace results
    trace_result_dict = {
        "Trace Number": i,
        "Trace Is Fit": trace_is_fit,
        "Trace Fitness": trace_fitness,
        "Missing Tokens": missing_tokens,
        "Consumed Tokens": consumed_tokens,
        "Remaining Tokens": remaining_tokens,
        "Produced Tokens": produced_tokens
    }

    # Append the trace result dictionary to the list
    trace_results_list.append(trace_result_dict)

    # Access other information as needed
    activated_transitions = trace_result['activated_transitions']
    reached_marking = trace_result['reached_marking']
    transitions_with_problems = trace_result['transitions_with_problems']
    enabled_transitions_in_marking = trace_result['enabled_transitions_in_marking']

# Define the path to the CSV file
csv_file_path = new_folder+"/trace_results.csv"

# Define the CSV fieldnames
fieldnames = ["Trace Number", "Trace Is Fit", "Trace Fitness", "Missing Tokens", "Consumed Tokens", "Remaining Tokens", "Produced Tokens"]

# Write the trace results to a CSV file with separators
with open(csv_file_path, mode='w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    
    # Write the header row
    writer.writeheader()
    
    for trace_result_dict in trace_results_list:
        # Write the trace results
        writer.writerow(trace_result_dict)
        
        # Write a separator row
        writer.writerow({key: '-' * 10 for key in fieldnames})

print(f"Trace results have been exported to {csv_file_path}")

Diagnostics (TBR)


"The execution of token-based replay in pm4py permits to obtain detailed information about transitions that did not execute correctly, or activities that are in the log and not in the model. In particular, executions that do not match the model are expected to take longer throughput time."

In [ ]:
from pm4py.algo.conformance.tokenreplay import algorithm as token_based_replay
net, initial_marking, final_marking = pm4py.discover_petri_net_inductive(log_inicial)
log_fail = log_converter.apply(events_fail)

parameters_tbr = {token_based_replay.Variants.TOKEN_REPLAY.value.Parameters.DISABLE_VARIANTS: True, token_based_replay.Variants.TOKEN_REPLAY.value.Parameters.ENABLE_PLTR_FITNESS: True}
replayed_traces, place_fitness, trans_fitness, unwanted_activities = token_based_replay.apply(log_fail, net,
                                                                                                initial_marking,
                                                                                                final_marking,
                                                                                                parameters=parameters_tbr)

print(trans_fitness)

csv_file_path = new_folder+"/trans_fitness.csv"

# Open the CSV file for writing and create a CSV writer
with open(csv_file_path, mode='w', newline='') as csv_file:
    fieldnames = ['Underfed_Traces', 'Fit_Traces']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    # Write the header row to the CSV file
    writer.writeheader()

    # Iterate through the trans_fitness dictionary and write each entry to the CSV file
    for fitness_data in trans_fitness.values():
        writer.writerow({
            'Underfed_Traces': str(fitness_data['underfed_traces']),
            'Fit_Traces': str(fitness_data['fit_traces'])
        })

Throughput analysis (unfit execution)

To perform throughput analysis on the transitions that were executed unfit, and then print on the console the result, the following code could be used:

Throughput analysis (activities)


To perform throughput analysis on the process executions containing activities that are not in the model, and then print the result on the screen, the following code could be used:

Alingments

In [ ]:
#net, initial_marking, final_marking = pm4py.discover_petri_net_inductive(log_no_timeout)
#aligned_traces = pm4py.conformance_diagnostics_alignments(log_fail, net, initial_marking, final_marking)
#pm4py.fitness_alignments(log_fail, net, initial_marking, final_marking)
#aligned_traces

In [ ]:
import os

from pm4py import util
from pm4py.algo.conformance import alignments as ali
from pm4py.algo.conformance.alignments.petri_net.variants.state_equation_a_star import Parameters
from pm4py.objects import log as log_lib
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.petri_net.importer import importer as petri_importer
from pm4py.objects.petri_net.utils.align_utils import pretty_print_alignments


def align(trace, net, im, fm, model_cost_function, sync_cost_function):
    trace_costs = list(map(lambda e: 1000, trace))
    params = dict()
    params[util.constants.PARAMETER_CONSTANT_ACTIVITY_KEY] = log_lib.util.xes.DEFAULT_NAME_KEY
    params[Parameters.PARAM_MODEL_COST_FUNCTION] = model_cost_function
    params[Parameters.PARAM_TRACE_COST_FUNCTION] = trace_costs
    params[Parameters.PARAM_SYNC_COST_FUNCTION] = sync_cost_function
    return ali.petri_net.algorithm.apply_trace(trace, net, im, fm, parameters=params,
                                   variant=ali.petri_net.algorithm.VERSION_STATE_EQUATION_A_STAR)


def execute_script():

    model_cost_function = dict()
    sync_cost_function = dict()
    for t in net.transitions:
        if t.label is not None:
            model_cost_function[t] = 1000
            sync_cost_function[t] = 0
        else:
            model_cost_function[t] = 1

    alignments = []
    for trace in log_fail:
        alignments.append(align(trace, net, initial_marking, final_marking, model_cost_function, sync_cost_function))

    pretty_print_alignments(alignments)


#execute_script()

Throughput Time
Para mirar la duración de cada caso

In [ ]:
all_case_durations = pm4py.get_all_case_durations(log_fail)
all_case_durations

Cycle Time and Waiting Time


In [ ]:
from pm4py.objects.log.util import interval_lifecycle
enriched_log = interval_lifecycle.assign_lead_cycle_time(log_fail)
enriched_log


Eventualy follows graph are a useful tool too

In [ ]:
efg_graph = pm4py.discover_eventually_follows_graph(log_fail)
efg_graph

Distribución de la duración de un caso

In [ ]:

from pm4py.util import constants
from pm4py.statistics.traces.generic.log import case_statistics
x, y = case_statistics.get_kde_caseduration(log_fail, parameters={constants.PARAMETER_CONSTANT_TIMESTAMP_KEY: "time:timestamp"})

from pm4py.visualization.graphs import visualizer as graphs_visualizer

gviz = graphs_visualizer.apply_plot(x, y, variant=graphs_visualizer.Variants.CASES)
graphs_visualizer.view(gviz)

gviz = graphs_visualizer.apply_semilogx(x, y, variant=graphs_visualizer.Variants.CASES)
#graphs_visualizer.view(gviz)
                                    

Distribution of events over time

In [ ]:
from pm4py.algo.filtering.log.attributes import attributes_filter


x, y = attributes_filter.get_kde_date_attribute(log_fail, attribute="time:timestamp")

from pm4py.visualization.graphs import visualizer as graphs_visualizer

gviz = graphs_visualizer.apply_plot(x, y, variant=graphs_visualizer.Variants.DATES)
graphs_visualizer.view(gviz)

Dotted Chart


In [ ]:
#pm4py.view_dotted_chart(log_fail, format="png")
                                    

In [ ]:
from pm4py.algo.discovery.batches import algorithm
batches = algorithm.apply(log_fail)
batches

Replay Fitness


In [ ]:
net, marking, fmarking = pm4py.discover_petri_net_inductive(log_no_timeout)

fitness = pm4py.fitness_alignments(log_fail, net, marking, fmarking)


# Initialize an empty string to build the dictionary representation
fitness_str = ""

# Build the string with spaces and line breaks between each element
for key, value in fitness.items():
    fitness_str += f"{key}: {value}\n"




In [ ]:
prec = pm4py.precision_alignments(log_fail, net, marking, fmarking)
prec = "precison: "+ str(prec)
prec

In [ ]:
from pm4py.algo.evaluation.generalization import algorithm as generalization_evaluator

gen = generalization_evaluator.apply(log_fail, net, marking, fmarking)
gen="generalization: " + str(gen)
gen


In [ ]:
from pm4py.algo.evaluation.simplicity import algorithm as simplicity_evaluator
simp = simplicity_evaluator.apply(net)
simp= "simplicity: " + str(simp)
simp

In [ ]:
# Save the string to a text file, overwriting the previous content
file_name = dir_tutti+"/log/Tesis/mision_variables/model_evaluation.txt"
with open(file_name, "w") as file:
    file.write(fitness_str)
    file.write(prec+"\n")
    file.write(gen+"\n")
    file.write(simp+"\n")


copy_file(file_name, new_folder)

